In [35]:
import numpy as np
import sympy as sp

In [36]:
# The look up table takes the form list[\sigma, ...] + list[Excitations]
# lookup_table = {
# 0: [],  # | c > b_0 b_0^\dagger b_0 b_0^\dagger
# 1: [0, 1],  # | \Phi_1^2 > +
# 2: [0, 2],  # | \Phi_1^3 > +
# 3: [1, 1],  # | \Phi_1^2 > -
# 4: [1, 2],  # | \Phi_1^3 > -
# 5: [0, 1, 1, 1],  # | \Phi_{1, 1}^{2, 2} >
# 6: [0, 1, 1, 2],  # | \Phi_{1, 1}^{2, 3} >
# 7: [0, 1, 2, 1],  # | \Phi_{1, 1}^{3, 2} >
# 8: [0, 1, 2, 2],  # | \Phi_{1, 1}^{3, 3} >
# }

In [37]:
# Encode the first number of the look up table as spin up or down, and the second as the excitation
lookup_table = {
    0: [],
    1: [0, 1],  # +
    2: [1, 1],  # -
    3: [0, 2],  # +
    4: [1, 2],  # -
}

In [38]:
from utils import read_elements, Z
from energy import Energy


groundstate = np.array(
    [
        [1, 0, 0],
        [1, 0, 0],
    ]
)
energy = Energy(groundstate)
values = energy.values


def antisymmetrize(alpha: int, beta: int, gamma: int, delta: int):
    return values[alpha, beta, gamma, delta] - values[alpha, beta, delta, gamma]


def annihilate_and_create(sigma: int, i: int, a: int):
    new_state = np.copy(groundstate)
    new_state[sigma, i] -= 1
    new_state[sigma, a] += 1
    return new_state

In [39]:
class SetupMatrix:
    def __init__(self, F: int, ref_energy: Energy):
        self.F = F
        self.groundstate = np.zeros((2, 3))
        self.groundstate[:, :F] = 1
        self.ref_energy = ref_energy
        self.values = self.ref_energy.values

    def get_hole_particle(
        self, state: np.ndarray
    ) -> tuple[tuple[int, int], tuple[int, int]]:
        hole = np.argwhere(state[:, : self.F] == 0)[0]
        particle = np.argwhere(state[:, self.F :] == 1)[0]
        particle = particle + np.array([0, self.F])

        return tuple(hole), tuple(particle)

    def energy_from_state(
        self, bra_state: np.ndarray, ket_state: np.ndarray
    ) -> sp.Expr:

        if np.all(bra_state == self.groundstate) and np.all(
            ket_state == self.groundstate
        ):
            return self.ref_energy.E_ref

        if np.all(bra_state == self.groundstate) or np.all(
            ket_state == self.groundstate
        ):
            acting = bra_state if np.all(ket_state == self.groundstate) else ket_state
            i, a = self.get_hole_particle(acting)

            return self.f(i, a)

        i, a = self.get_hole_particle(bra_state)
        j, b = self.get_hole_particle(ket_state)

        energy = self.ref_energy.E_ref * self.delta(i, j) * self.delta(a, b)
        energy += self.f(a, b) * self.delta(i, j)
        energy -= self.f(i, j) * self.delta(a, b)
        energy += self.antisymmetrized(a, j, i, b)

        return energy

    def delta(self, alpha: int, beta: int) -> int:
        if alpha == beta:
            return 1
        return 0

    def h0(self, p: tuple[int, int], q: tuple[int, int]) -> sp.Expr:
        if p != q:
            return 0
        n = p[1]
        return -(Z**2) / (2 * (n + 1) ** 2)

    def v(
        self,
        p: tuple[int, int],
        q: tuple[int, int],
        r: tuple[int, int],
        s: tuple[int, int],
    ) -> sp.Expr:
        spins = tuple(map(lambda x: x[0], [p, q, r, s]))
        levels = tuple(map(lambda x: x[1], [p, q, r, s]))
        if not self.spin_ok(*spins):
            return 0

        if p == q or r == s:
            return 0

        return self.values[levels[0], levels[1], levels[2], levels[3]]

    def antisymmetrized(
        self,
        p: tuple[int, int],
        q: tuple[int, int],
        r: tuple[int, int],
        s: tuple[int, int],
    ) -> sp.Expr:
        return self.v(p, q, r, s) - self.v(p, q, s, r)

    def f(self, p: tuple[int, int], q: tuple[int, int]):
        energy = self.h0(p, q)

        for spin in range(2):
            for k in range(self.F):
                energy += self.antisymmetrized(p, (spin, k), q, (spin, k))

        return energy

    def spin_ok(self, a: int, b: int, c: int, d: int) -> bool:
        if a == c and b == d:
            return True
        return False

In [46]:
total_states = [groundstate]

for a in range(1, 3):
    for sigma in range(2):
        print(f"Annihilate {0} and create {a} with spin {sigma}")
        new_state = annihilate_and_create(sigma, 0, a)
        total_states.append(new_state)

Annihilate 0 and create 1 with spin 0
Annihilate 0 and create 1 with spin 1
Annihilate 0 and create 2 with spin 0
Annihilate 0 and create 2 with spin 1


In [47]:
Hamiltonian = sp.zeros(len(total_states), len(total_states))
setup = SetupMatrix(1, energy)

for i, bra_state in enumerate(total_states):
    for j, ket_state in enumerate(total_states):
        Hamiltonian[i, j] = setup.energy_from_state(bra_state, ket_state)

In [64]:
he_Hamiltonian = Hamiltonian.subs(Z, 2)
he_Hamiltonian.evalf()

Matrix([
[             -2.75,  0.178710066838823,  0.178710066838823, 0.0879188989921962, 0.0879188989921962],
[ 0.178710066838823,  -2.08024691358025, 0.0438957475994513,  0.101052953825573, 0.0224458369965822],
[ 0.178710066838823, 0.0438957475994513,  -2.08024691358025, 0.0224458369965822,  0.101052953825573],
[0.0879188989921962,  0.101052953825573, 0.0224458369965822,  -2.02324761284722,   0.01153564453125],
[0.0879188989921962, 0.0224458369965822,  0.101052953825573,   0.01153564453125,  -2.02324761284722]])

In [67]:
he_eigenvals = [
    sp.re(key.evalf())
    for key, value in he_Hamiltonian.eigenvals().items()
    for _ in range(value)
]

In [71]:
lmbda = min(he_eigenvals)
lmbda

-2.83864845277878

In [70]:
lmbda * 2 * 13.6

-77.2112379155829